Non-Linear Curve Fitting, Part 1
=========================

<div class="overview-this-is-a-title overview">
<p class="overview-title">Overview</p>
<p>Questions</p>
    <ul>
        <li>How can I analyze enzyme kinetics data in Python?</li>
        <li>What is the process for non-linear least squares curve fitting in Python?</li>
    </ul>
<p>Objectives:</p>
    <ul>
        <li> Create a pandas dataframe with enzyme kinetics data from a .csv file</li>
        <li> Add velocity calculations to the dataframe</li>
        <li> Perform the non-linear regression calculations</li>
    </ul>
</div>

In [1]:
# import the libraries we need
import os # to create a filehandle for the .csv file
import pandas as pd # for importing the .csv file and creating a dataframe
import numpy as np # for calculations and datatyping. ***Jessica - is this necessary?***
from scipy import stats # for performing non-linear regression

In [2]:
cd ../..

/Users/pac8612/Desktop/python-scripting-biochemistry


In [3]:
datafile = os.path.join('biochemist-python', 'chapters', 'data', 'AP_kinetics.csv') # filehandle created
print(datafile)  # filehandle confirmed

biochemist-python/chapters/data/AP_kinetics.csv


In [4]:
AP_kinetics_df = pd.read_csv(datafile)  # Use pandas to create a dataframe of the alkaline phosphatase kinetics data
AP_kinetics_df  # dataframe confirmed

,pNPP (mM),0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,...,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75,5
0,20.00,0.073923,0.139234,0.226077,0.287081,0.366029,0.434928,0.522488,0.574163,0.671770,...,0.828947,0.818182,0.933014,1.044976,1.098086,1.182775,1.256699,1.266029,1.431818,1.392344
1,10.00,0.066055,0.143119,0.208486,0.264220,0.330275,0.396330,0.481651,0.522936,0.606881,...,0.794725,0.784404,0.849771,0.915138,1.042431,1.067890,1.193119,1.250917,1.294266,1.444954
2,7.00,0.063797,0.130253,0.205348,0.255190,0.328956,0.394747,0.455886,0.515696,0.610063,...,0.738323,0.789494,0.889842,0.911772,0.986867,1.105823,1.095854,1.244051,1.325791,1.262658
3,4.00,0.060612,0.121224,0.192857,0.237551,0.303061,0.367347,0.441429,0.499592,0.567551,...,0.666735,0.720000,0.764082,0.848571,0.881633,0.950204,1.061633,1.113061,1.186531,1.175510
4,2.00,0.052759,0.104483,0.147414,0.215172,0.271552,0.322759,0.372931,0.409655,0.465517,...,0.568966,0.614483,0.652241,0.753103,0.744828,0.786207,0.861724,0.921724,1.012241,1.075862
5,1.00,0.037895,0.080526,0.112500,0.165789,0.187500,0.232105,0.273553,0.318947,0.348158,...,0.434211,0.454737,0.538816,0.574737,0.580263,0.663158,0.677763,0.703421,0.712500,0.821053
6,0.70,0.033797,0.067594,0.093516,0.127312,0.170625,0.206719,0.229687,0.257250,0.307125,...,0.364547,0.381937,0.426563,0.477750,0.516797,0.519750,0.541078,0.561094,0.592266,0.669375
7,0.40,0.023538,0.044308,0.068538,0.093231,0.116538,0.142615,0.159923,0.192000,0.209769,...,0.243692,0.279692,0.309000,0.329538,0.360000,0.358154,0.396231,0.402923,0.442846,0.443077
8,0.20,0.012955,0.027000,0.042955,0.055636,0.066136,0.077727,0.095455,0.108000,0.123955,...,0.157500,0.165273,0.179045,0.194727,0.196364,0.229091,0.241091,0.245455,0.269455,0.286364
9,0.10,0.007350,0.014700,0.022500,0.028800,0.036750,0.045450,0.050400,0.061800,0.066825,...,0.080850,0.085500,0.092625,0.100800,0.117000,0.122400,0.122400,0.135000,0.145350,0.142500


### Datatype
Now that we have imported our date, we need to check the datatypes for the numbers. We must ensure that the numbers are floats, rather than strings, so we can do calculations on them.

Notice that the df.dtypes command gives the overall datatype for the dataframe as an `object`, but also lists the datatypes for each of the columns.

In [5]:
AP_kinetics_df.dtypes # checking to see if the numbers are strings or floats

pNPP (mM)    float64
0.25         float64
0.5          float64
0.75         float64
1            float64
1.25         float64
1.5          float64
1.75         float64
2            float64
2.25         float64
2.5          float64
2.75         float64
3            float64
3.25         float64
3.5          float64
3.75         float64
4            float64
4.25         float64
4.5          float64
4.75         float64
5            float64
dtype: object

In [6]:
display(list(AP_kinetics_df.columns.values)) # checking to see if the column labels are strings or floats

['pNPP (mM)',
 '0.25',
 '0.5',
 '0.75',
 '1',
 '1.25',
 '1.5',
 '1.75',
 '2',
 '2.25',
 '2.5',
 '2.75',
 '3',
 '3.25',
 '3.5',
 '3.75',
 '4',
 '4.25',
 '4.5',
 '4.75',
 '5']

### Calculating initial velocities

The first column in our dataframe is the pNPP concentration in mM ('pNPP (mM)'). The other colulmn headers are the times in minutes for the kinetic data. Notice that these are listed as strings. To calculate initial velocities, these need to be changed to floats.

We need to set up the column headers as our x values. For the y values, we need to skip the first value ('pNPP (mM)') and then use the remaining values (A-405 as a function of time) to calculate slopes and get our initial velocities. The extinction coefficient for p-nitrophenol under these buffer conditions is 15.0 mM<sup>-1</sup>cm<sup>-1</sup>.

In [7]:
AP_kinetics_df.columns.values[1:].astype('float64')

array([0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 , 2.75,
       3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75, 5.  ])

In [8]:
xdata = AP_kinetics_df.columns.values[1:].astype('float64') # extracting the column headers as floats
print(xdata)

[0.25 0.5  0.75 1.   1.25 1.5  1.75 2.   2.25 2.5  2.75 3.   3.25 3.5
 3.75 4.   4.25 4.5  4.75 5.  ]


In [9]:
# AP_kinetics_df.drop(columns = 'pNPP (mM)', inplace=True) - syntax for dropping a column. Had to do this repeatedly
# AP_kinetics_df

In [10]:
AP_kinetics_df.iloc[0,1:] # Learn to extract the A-405 values for the first concentration

0.25    0.073923
0.5     0.139234
0.75    0.226077
1       0.287081
1.25    0.366029
1.5     0.434928
1.75    0.522488
2       0.574163
2.25    0.671770
2.5     0.724880
2.75    0.828947
3       0.818182
3.25    0.933014
3.5     1.044976
3.75    1.098086
4       1.182775
4.25    1.256699
4.5     1.266029
4.75    1.431818
5       1.392344
Name: 0, dtype: float64

In [11]:
ydata = AP_kinetics_df.iloc[0, 1:]
print(ydata)

0.25    0.073923
0.5     0.139234
0.75    0.226077
1       0.287081
1.25    0.366029
1.5     0.434928
1.75    0.522488
2       0.574163
2.25    0.671770
2.5     0.724880
2.75    0.828947
3       0.818182
3.25    0.933014
3.5     1.044976
3.75    1.098086
4       1.182775
4.25    1.256699
4.5     1.266029
4.75    1.431818
5       1.392344
Name: 0, dtype: float64


In [12]:
def slope_only(xdata, ydata):  # SciPy linregress has five outputs; I only want the slope
    slope, intercept, rvalue, pvalue, stderr = stats.linregress(xdata, ydata)
    return slope

slope = slope_only(xdata,ydata)
print(slope)

0.2892506385894737


In [13]:
slope_list = []  # setting up a list to contain the slope values
for i in range(0, len(AP_kinetics_df)):  # looping through the pandas dataframe. Is there a better way to do this?
    xdata = AP_kinetics_df.columns.values[1:21].astype('float64')
    ydata = AP_kinetics_df.iloc[i, 1:]
    slope = slope_only(xdata, ydata)
    slope_list.append(slope)
print(slope_list)

[0.2892506385894737, 0.27880733943157904, 0.26825116593082704, 0.24139941690526318, 0.20673321230977446, 0.1595900277142857, 0.12890131578947372, 0.09117987271278195, 0.05658209154887218, 0.02960706766917293, 0.015764305515789476]


These slope values are correct for each row in the dataframe. Now I will add them to a new dataframe. Next I'll use the extinction coefficient for p-nitrophenol (0.015/$\mu$M/min) to create a second column where the initial velocity is given in mM/min.

In [14]:
MM_df = pd.DataFrame(AP_kinetics_df, columns = ['pNPP (mM)']) # creating a new dataframe with the first column of the old dataframe
MM_df['slopes'] = slope_list  # adding slopes to the dataframe
MM_df['Initial Velocities'] = MM_df['slopes'] / 0.015 # adding the initial velocities

In [15]:
MM_df

,pNPP (mM),slopes,Initial Velocities
0,20.00,0.289251,19.283376
1,10.00,0.278807,18.587156
2,7.00,0.268251,17.883411
3,4.00,0.241399,16.093294
4,2.00,0.206733,13.782214
5,1.00,0.159590,10.639335
6,0.70,0.128901,8.593421
7,0.40,0.091180,6.078658
8,0.20,0.056582,3.772139
9,0.10,0.029607,1.973805


We will use this dataframe now to perform the nonlinear regression fit using the SciPy library in part 2 of this lesson. To save this data for part 2, so we need to write it to a csv file in our data directory.

In [16]:
MM_df.to_csv('biochemist-python/chapters/data/MM_data.csv')

<div class="exercise-this-is-a-title exercise">
<p class="exercise-title">Check your understanding</p>
    <p>You will find an Excel file in your data folder, chymotrypsin_kinetics.xlsx, with some kinetic data from a chymotrypsin experiment. Apply the principles above to create dataframes and a .csv file for creating a Michaelis-Menten plot with these data. Under these assay conditions the extinction coefficient for p-nitrophenol is 18,320 M<sup>-1</sup>cm<sup>-1</sup>.</p>

```{admonition} Hint
:class: dropdown
    You will need to get the data into a layout and file format that is easily read by pandas. 
    <ul>
        <li>Delete the first seven lines of the Excel file.</li>
        <li>Delete the first column of the Excel file.</li>
        <li>Save the file as chymotrypsin_kinetics.csv.</li>
        <li>Your data will should look something like this:</li>
        <img src="biochemist-python/chapters/images/csv_image.png" alt="csv image">
```   
```{admonition} Solution
:class: dropdown
    
```python
        import os 
        import pandas as pd 
        import numpy as np 
        from scipy import stats 
        datafile = os.path.join('biochemist-python', 'chapters', 'data', 'chymotrypsin_kinetics.csv')
        chymo_rates_df = pd.read_csv(datafile)
        
        def slope_only(xdata, ydata): 
            slope, intercept, rvalue, pvalue, stderr = stats.linregress(xdata, ydata)
            return slope
        
        slope_list = [] 
        for i in range(0, len(chymo_rates_df)):
            xdata = chymo_rates_df.columns.values[2:len(chymo_rates_df.columns)].astype('float64')
            ydata = chymo_rates_df.iloc[i, 2:len(chymo_rates_df.columns)]
            slope = slope_only(xdata, ydata)
            slope_list.append(slope)

        chymo_MM_df = pd.DataFrame(chymo_rates_df, columns = ['[pNPA] (mM)'])      
        chymo_MM_df['slopes'] = slope_list 
        chymo_MM_df['Initial Velocities'] = MM_df['slopes'] / 18.32 
        MM_df.to_csv('biochemist-python/chapters/data/chymo_MM_data.csv')
``` 
</div>


In [17]:
import os 
import pandas as pd 
import numpy as np 
from scipy import stats 
datafile = os.path.join('biochemist-python', 'chapters', 'data', 'chymotrypsin_kinetics.csv') # filehandle created
chymo_rates_df = pd.read_csv(datafile)

def slope_only(xdata, ydata):  # SciPy linregress has five outputs; I only want the slope
    slope, intercept, rvalue, pvalue, stderr = stats.linregress(xdata, ydata)
    return slope

slope_list = []  # setting up a list to contain the slope values
for i in range(0, len(chymo_rates_df)):  # looping through the pandas dataframe. Is there a better way to do this?
    xdata = chymo_rates_df.columns.values[2:len(chymo_rates_df.columns)].astype('float64')
    ydata = chymo_rates_df.iloc[i, 2:len(chymo_rates_df.columns)]
    slope = slope_only(xdata, ydata)
    slope_list.append(slope)

chymo_MM_df = pd.DataFrame(chymo_rates_df, columns = ['[pNPA] (mM)'])      
chymo_MM_df
chymo_MM_df['slopes'] = slope_list 
chymo_MM_df['Initial Velocities'] = MM_df['slopes'] / 18.32 
chymo_MM_df
MM_df.to_csv('biochemist-python/chapters/data/chymo_MM_data.csv')
